In [6]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split

from tensorflow.keras import models, layers

from sklearn.metrics import accuracy_score

In [ ]:
!wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz

In [3]:
!mkdir ./cifar_10/
!tar -xf cifar-10-python.tar.gz -C ./cifar_10/

mkdir: cannot create directory ‘./cifar_10/’: File exists


In [4]:
!ls -l ./cifar_10/cifar-10-batches-py/

total 181876
-rw-r--r-- 1 2156 1103      158 Mar 31  2009 batches.meta
-rw-r--r-- 1 2156 1103 31035704 Mar 31  2009 data_batch_1
-rw-r--r-- 1 2156 1103 31035320 Mar 31  2009 data_batch_2
-rw-r--r-- 1 2156 1103 31035999 Mar 31  2009 data_batch_3
-rw-r--r-- 1 2156 1103 31035696 Mar 31  2009 data_batch_4
-rw-r--r-- 1 2156 1103 31035623 Mar 31  2009 data_batch_5
-rw-r--r-- 1 2156 1103       88 Jun  4  2009 readme.html
-rw-r--r-- 1 2156 1103 31035526 Mar 31  2009 test_batch


In [7]:
workdir = os.curdir
datadir = os.path.join(workdir,'cifar_10','cifar-10-batches-py')
train_files = [os.path.join(datadir,f'data_batch_{i}') for i in range(1,6)]
test_file = 'test_batch'

In [8]:
# unpickle the data file containing the image
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [9]:
data_dict = unpickle(train_files[0])
data_dict.keys()

dict_keys([b'batch_label', b'labels', b'data', b'filenames'])

In [10]:
# read all files into a dataset
data = np.empty(data_dict[b'data'].shape)
labels = list()
for fl in train_files:
  print(f"Loading File {fl}")
  df = unpickle(fl)
  print(df[b'batch_label'])
  print(f"Geting Data")
  if df[b'batch_label'] == b'training batch 1 of 5':
    data = df[b'data']
  else:
    data = np.concatenate([data,df[b'data']])
  print("Geting labels")
  labels = labels + df[b'labels']
  print(f"Shape of Data is {data.shape}")
  print("="*50)

Loading File ./cifar_10/cifar-10-batches-py/data_batch_1
b'training batch 1 of 5'
Geting Data
Geting labels
Shape of Data is (10000, 3072)
Loading File ./cifar_10/cifar-10-batches-py/data_batch_2
b'training batch 2 of 5'
Geting Data
Geting labels
Shape of Data is (20000, 3072)
Loading File ./cifar_10/cifar-10-batches-py/data_batch_3
b'training batch 3 of 5'
Geting Data
Geting labels
Shape of Data is (30000, 3072)
Loading File ./cifar_10/cifar-10-batches-py/data_batch_4
b'training batch 4 of 5'
Geting Data
Geting labels
Shape of Data is (40000, 3072)
Loading File ./cifar_10/cifar-10-batches-py/data_batch_5
b'training batch 5 of 5'
Geting Data
Geting labels
Shape of Data is (50000, 3072)


In [11]:
# Reading Metadata
label_meta = unpickle(os.path.join(datadir,'batches.meta'))[b'label_names']
label_meta_mapping = {i:label_meta[i].decode('utf-8') for i in range(len(label_meta))}
label_meta_mapping

{0: 'airplane',
 1: 'automobile',
 2: 'bird',
 3: 'cat',
 4: 'deer',
 5: 'dog',
 6: 'frog',
 7: 'horse',
 8: 'ship',
 9: 'truck'}

In [12]:
labels = np.array(labels)
for i in range(0,10):
  lb_count = np.sum((labels == i)*1)
  print(f"{label_meta_mapping[i]}: {lb_count}")

airplane: 5000
automobile: 5000
bird: 5000
cat: 5000
deer: 5000
dog: 5000
frog: 5000
horse: 5000
ship: 5000
truck: 5000


In [ ]:
np.sqrt(3072)

55.42562584220407

In [65]:
data_rgb = np.reshape(data, (data.shape[0], 32, 32, 3))

In [66]:
train_X, test_X, train_y, test_y = train_test_split(data_rgb, np.array(labels), test_size=0.2)
test_X, val_X, test_y, val_y = train_test_split(test_X, test_y, test_size=0.7)
print(f"Training Data Size : {train_X.shape[0]}")
print(f"Validation Data Size : {val_X.shape[0]}")
print(f"Testing Data Size : {test_X.shape[0]}")

Training Data Size : 40000
Validation Data Size : 7000
Testing Data Size : 3000


In [67]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                6

In [68]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [69]:
model.fit(x=train_X, y=train_y, batch_size=32, epochs=10, validation_data=(val_X, val_y), workers=-1,use_multiprocessing=True)

Epoch 1/10
1250/1250 [==============================] - 4s 3ms/step - loss: 2.8516 - accuracy: 0.2491 - val_loss: 1.6449 - val_accuracy: 0.4043
Epoch 2/10
1250/1250 [==============================] - 4s 3ms/step - loss: 1.6358 - accuracy: 0.4099 - val_loss: 1.5614 - val_accuracy: 0.4417
Epoch 3/10
1250/1250 [==============================] - 4s 3ms/step - loss: 1.5568 - accuracy: 0.4430 - val_loss: 1.5458 - val_accuracy: 0.4497
Epoch 4/10
1250/1250 [==============================] - 3s 3ms/step - loss: 1.4792 - accuracy: 0.4790 - val_loss: 1.5587 - val_accuracy: 0.4443
Epoch 5/10
1250/1250 [==============================] - 4s 3ms/step - loss: 1.4299 - accuracy: 0.4920 - val_loss: 1.4681 - val_accuracy: 0.4794
Epoch 6/10
1250/1250 [==============================] - 4s 3ms/step - loss: 1.3846 - accuracy: 0.5103 - val_loss: 1.4417 - val_accuracy: 0.4821
Epoch 7/10
1250/1250 [==============================] - 3s 3ms/step - loss: 1.3431 - accuracy: 0.5241 - val_loss: 1.5155 - val_accuracy:

In [22]:
def top_5_error_rate(predict_proba, y_true):
  result = 0
  for i in range(len(y_true)):
    row = predict_proba[i,:]
    top_5 = sorted(range(len(row)), key = lambda x: row[x], reverse=True)[:5]
    if y_true[i] in top_5:
      result = result + 1
  
  return result/len(y_true)

In [70]:
proba_test_X = model.predict(test_X)
print(f'Accuracy Score : {model.evaluate(test_X, test_y)}')
print(top_5_error_rate(proba_test_X, test_y))

94/94 [==============================] - 0s 2ms/step - loss: 1.4009 - accuracy: 0.5110
Accuracy Score : [1.4008930921554565, 0.5109999775886536]
0.9263333333333333


In [30]:
proba_test_X.shape

(3000, 10)

In [82]:
test_batch = unpickle(os.path.join(datadir,'test_batch'))
test_data = test_batch[b'data']
test_labels = test_batch[b'labels']

In [83]:
test_data = np.reshape(test_data, (test_data.shape[0], 32, 32, 3))

In [84]:
proba_train_X = model.predict(test_data)
print(f'Accuracy Score : {model.evaluate(test_data, np.array(test_labels))}')
print(top_5_error_rate(proba_train_X, test_labels))

313/313 [==============================] - 1s 2ms/step - loss: 1.3740 - accuracy: 0.5154
Accuracy Score : [1.374024748802185, 0.5153999924659729]
0.9342


In [31]:
def preprocess_image(images):
  norm_image = images.astype('float32')
  norm_image = norm_image/255.0

  return norm_image

In [32]:
norm_rgb_image = preprocess_image(data_rgb)

In [53]:
train_X, val_X, train_y, val_y = train_test_split(norm_rgb_image, np.array(labels), test_size=0.3)
#test_X, val_X, test_y, val_y = train_test_split(test_X, test_y, test_size=0.7)
print(f"Training Data Size : {train_X.shape[0]}")
print(f"Validation Data Size : {val_X.shape[0]}")
#print(f"Testing Data Size : {test_X.shape[0]}")

Training Data Size : 35000
Validation Data Size : 15000


In [59]:
def model_2():
  model = models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='RandomNormal', padding='same', input_shape=(32, 32, 3)))
  model.add(layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='RandomNormal', padding='same'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Dropout(0.2))
  model.add(layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='RandomNormal', padding='same'))
  model.add(layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='RandomNormal', padding='same'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Dropout(0.2))
  model.add(layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='RandomNormal', padding='same'))
  model.add(layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='RandomNormal', padding='same'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Dropout(0.2))
  model.add(layers.Flatten())
  model.add(layers.Dense(128, activation='relu', kernel_initializer='RandomNormal'))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(10, activation='softmax'))
  # compile model
  #opt = SGD(lr=0.001, momentum=0.9)
  model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
  return model

In [60]:
model2 = model_2()
model.fit(x=train_X, y=train_y, batch_size=64, epochs=10, validation_data=(val_X, val_y), workers=-1,use_multiprocessing=True)

Epoch 1/10
547/547 [==============================] - 3s 5ms/step - loss: 0.4839 - accuracy: 0.8322 - val_loss: 1.0199 - val_accuracy: 0.6796
Epoch 2/10
547/547 [==============================] - 3s 5ms/step - loss: 0.4662 - accuracy: 0.8398 - val_loss: 1.0870 - val_accuracy: 0.6723
Epoch 3/10
547/547 [==============================] - 3s 5ms/step - loss: 0.4544 - accuracy: 0.8423 - val_loss: 1.0947 - val_accuracy: 0.6731
Epoch 4/10
547/547 [==============================] - 3s 5ms/step - loss: 0.4389 - accuracy: 0.8489 - val_loss: 1.1470 - val_accuracy: 0.6603
Epoch 5/10
547/547 [==============================] - 2s 5ms/step - loss: 0.4270 - accuracy: 0.8513 - val_loss: 1.1729 - val_accuracy: 0.6667
Epoch 6/10
547/547 [==============================] - 3s 5ms/step - loss: 0.4157 - accuracy: 0.8567 - val_loss: 1.1923 - val_accuracy: 0.6678
Epoch 7/10
547/547 [==============================] - 3s 5ms/step - loss: 0.3957 - accuracy: 0.8633 - val_loss: 1.2161 - val_accuracy: 0.6624
Epoch 

In [89]:
proba_val_X = model2.predict(val_X)
print(f'Accuracy Score : {model2.evaluate(val_X, val_y)}')
print(top_5_error_rate(proba_val_X, val_y))

219/219 [==============================] - 0s 2ms/step - loss: 42.4731 - accuracy: 0.0826
Accuracy Score : [42.47309112548828, 0.08257143199443817]
0.48328571428571426


In [85]:
test_data_norm = preprocess_image(test_data)

In [1]:
proba_test_X = model2.predict(test_data)
print(f'Accuracy Score : {model2.evaluate(test_data, np.array(test_labels))}')
print(top_5_error_rate(proba_test_X, test_labels))

NameError: ignored

In [94]:
test_data_norm.shape

(10000, 32, 32, 3)